<a href="https://colab.research.google.com/github/arunraja-hub/Preference_Extraction/blob/master/export_tf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The first part sets up the model to export it in tf 2.0 format. The "Restore and verify it worked." part shows how to import the tf 2.0 version. That code can be run by itself to import the [version on github.](https://github.com/arunraja-hub/Preference_Extraction/tree/master/saved_model2)

#Install and imports

In [11]:
%tensorflow_version 2.x

!git clone https://github.com/arunraja-hub/Preference_Extraction.git

!pip install tf-agents==0.3.0

!pip uninstall tensorflow-probability -y
!pip install tensorflow-probability==0.7.0

import tensorflow as tf

fatal: destination path 'Preference_Extraction' already exists and is not an empty directory.
Uninstalling tensorflow-probability-0.7.0:
  Successfully uninstalled tensorflow-probability-0.7.0
  Using cached https://files.pythonhosted.org/packages/3e/3a/c10b6c22320531c774402ac7186d1b673374e2a9d12502cbc8d811e4601c/tensorflow_probability-0.7.0-py2.py3-none-any.whl


In [0]:
import numpy as np

from tf_agents.trajectories.time_step import TimeStep
from tf_agents.specs.tensor_spec import TensorSpec
from tf_agents.specs.tensor_spec import TensorSpec
from tf_agents.specs.tensor_spec import BoundedTensorSpec
from tf_agents.networks import q_network

import concurrent.futures
import itertools
import os
import pickle
import random
import sys
import time

import numpy as np
import io
import collections

# Setup model and restore

In [0]:
cpt_name = "Preference_Extraction/model_ckpt"

In [0]:
input_shape = [14, 16, 5]
q_net = q_network.QNetwork(input_tensor_spec=TensorSpec(shape=input_shape), action_spec=BoundedTensorSpec((), tf.int32, 0, 2), conv_layer_params = [[16, 3, 1], [32, 3, 2]], fc_layer_params = [64])
q_net.layers[0].layers[1]._name = "EncodingNetwork/conv2d_1"

In [0]:
latest_cpt =  tf.train.latest_checkpoint(cpt_name)
reader = tf.compat.v1.train.NewCheckpointReader(latest_cpt)

In [0]:
model_input = tf.keras.Input(shape=input_shape)
q_model_nested = tf.keras.models.Model(inputs=model_input, outputs=[q_net(model_input)])
q_model_nested.build(input_shape=input_shape)

In [0]:
def flatten_model(model_nested):
    def get_layers(layers):
        layers_flat = []
        for layer in layers:
            try:
                layers_flat.extend(get_layers(layer.layers))
            except AttributeError:
                layers_flat.append(layer)
        return layers_flat

    model_flat = tf.keras.models.Sequential(
        get_layers(model_nested.layers)
    )
    return model_flat

q_model = flatten_model(q_model_nested)

In [18]:
q_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
EncodingNetwork/conv2d (Conv (None, 12, 14, 16)        736       
_________________________________________________________________
EncodingNetwork/conv2d_1 (Co (None, 5, 6, 32)          4640      
_________________________________________________________________
flatten (Flatten)            (None, 960)               0         
_________________________________________________________________
EncodingNetwork/dense (Dense (None, 64)                61504     
_________________________________________________________________
dense (Dense)                (None, 3)                 195       
Total params: 67,075
Trainable params: 67,075
Non-trainable params: 0
_________________________________________________________________


In [0]:
layer_map = {
    q_model.layers[0]: "agent/_q_network/_encoder/_postprocessing_layers/0",
    q_model.layers[1]: "agent/_q_network/_encoder/_postprocessing_layers/1",
    q_model.layers[3]: "agent/_q_network/_encoder/_postprocessing_layers/3",
    q_model.layers[4]: "agent/_q_network/_q_value_layer",
}

In [0]:
last_name_part = "/.ATTRIBUTES/VARIABLE_VALUE"
for keras_layer, weights_bias_name in layer_map.items():
  weights = reader.get_tensor(weights_bias_name+"/kernel"+last_name_part)
  biases = reader.get_tensor(weights_bias_name+"/bias"+last_name_part)
  keras_layer.set_weights([weights, biases])

# Read data

In [0]:
class Trajectory(
    collections.namedtuple('Trajectory', [
        'step_type',
        'observation',
        'action',
        'policy_info',
        'next_step_type',
        'reward',
        'discount',
    ])):
  """Stores the observation the agent saw and the action it took.
      The rest of the attributes aren't used in this code."""
  __slots__ = ()

class ListWrapper(object):
  def __init__(self, list_to_wrap):
    self._list = list_to_wrap

  def as_list(self):
    return self._list

class RenameUnpickler(pickle.Unpickler):
    def find_class(self, module, name):
      if name == "Trajectory":
        return Trajectory
      if name == "ListWrapper":
        return ListWrapper

      return super(RenameUnpickler, self).find_class(module, name)

def rename_load(s):
    """Helper function analogous to pickle.loads()."""
    return RenameUnpickler(s, encoding='latin1').load()

In [0]:
# Modified read trajectories functions to read files from local storage

def load_file(full_path):
    try:
        with open(full_path, 'rb') as f:
            data = rename_load(f)
            return data
    except:
        return None
    
def all_load_data(base_path):
    
    executor = concurrent.futures.ThreadPoolExecutor(max_workers=100)
    
    futures = []
    for i in range(5000):
        full_path = os.path.join(base_path, "ts"+str(i)+".pickle")
        future = executor.submit(load_file, full_path)
        futures.append(future)
    
    raw_data = []
    for future in concurrent.futures.as_completed(futures):
        result = future.result()
        if result:
            raw_data.append(result)
    
    return raw_data


all_raw_data = all_load_data("Preference_Extraction/data/simple_env_1/")

# Verify the model does inference correctly

In [23]:
def verify_model(model):
  activation_model = tf.keras.models.Model(inputs=model.input, outputs=model.layers[-2].output)
  for i in range(len(all_raw_data[0].observation)):
    single_observation = np.array([all_raw_data[0].observation[i]])

    restored_activations = activation_model(single_observation)[0]
    old_activations = all_raw_data[0].policy_info["activations"][i]

    if i < 3:
      print("restored_activations", restored_activations, "old_activations", old_activations)

    np.testing.assert_allclose(restored_activations, old_activations, rtol=.1)

verify_model(q_model)

restored_activations tf.Tensor(
[ 0.        0.        0.        0.        0.        0.        0.
 68.69019   0.        0.        0.       37.881477  0.       68.34023
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.       55.817192  0.        0.        0.        0.        0.
  0.        0.       24.650705  0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        8.948536  0.
  0.      ], shape=(64,), dtype=float32) old_activations [ 0.        0.        0.        0.        0.        0.        0.
 68.6902    0.        0.        0.       37.88143   0.       68.340225
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0. 

# Export in tf 2.0 format

In [0]:
new_save_path = "Preference_Extraction/saved_model2"

In [25]:
q_model.save(new_save_path, overwrite=True)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: Preference_Extraction/saved_model2/assets


# Restore and verify it worked.

In [26]:
restored_model = tf.keras.models.load_model(new_save_path)
restored_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
EncodingNetwork/conv2d (Conv (None, 12, 14, 16)        736       
_________________________________________________________________
EncodingNetwork/conv2d_1 (Co (None, 5, 6, 32)          4640      
_________________________________________________________________
flatten (Flatten)            (None, 960)               0         
_________________________________________________________________
EncodingNetwork/dense (Dense (None, 64)                61504     
_________________________________________________________________
dense (Dense)                (None, 3)                 195       
Total params: 67,075
Trainable params: 67,075
Non-trainable params: 0
_________________________________________________________________


In [27]:
verify_model(restored_model)

restored_activations tf.Tensor(
[ 0.        0.        0.        0.        0.        0.        0.
 68.69019   0.        0.        0.       37.881477  0.       68.34023
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.       55.817192  0.        0.        0.        0.        0.
  0.        0.       24.650705  0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        8.948536  0.
  0.      ], shape=(64,), dtype=float32) old_activations [ 0.        0.        0.        0.        0.        0.        0.
 68.6902    0.        0.        0.       37.88143   0.       68.340225
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0. 